In [1]:
import torch
import torch.nn.functional as F
import os
from torch import nn
import pandas as pd
from torch.nn import BatchNorm1d, ReLU, Linear, Sequential
import numpy as np
from torch_geometric.data import Data, InMemoryDataset, download_url
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv, GINConv, global_add_pool
from torch_geometric.loader import DataLoader
import networkx as nx
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from src.models.model import *
from src.models.train import *
from src.data.netlist import *
import pickle

from persim import PersImage
from persim import PersistenceImager

In [2]:
%load_ext autoreload

%autoreload 2

design = "aes"

In [4]:
design = "aes"
for path in os.listdir(f"/home/zluo/data/graphs/{design}/"):
    file = path.split(".")[0]
    print(f"processing {file}")
    adj = np.load(f"/home/zluo/data/graphs/{design}/{file}.npy")
    G = nx.DiGraph(adj.T.tolist())
    # LOAD GLOBAL FEATURES
    # num of inst, num of nets, avg pins per inst, avg pins per net, number of seq, number of inst, number of bTerms
    extra_arr = np.load(f"/home/zluo/data/output/{design}/{file}_extra.npy")
    # type of gates
    with open(f'/home/zluo/data/output/{design}/type_dict_{file}.pkl', 'rb') as f:
        type_dict = pickle.load(f)

    # inst names
    with open(f'/home/zluo/data/output/{design}/{file}inst.pkl', 'rb') as f:
        inst_dict = pickle.load(f)
        type_node_dict = dict()
        seq_nodes = []
        for k, v in type_dict.items():
            node = inst_dict[k]
            if v == 'seq':
                seq_nodes.append(node)
            type_node_dict[node] = v
            
    with open(f'/home/zluo/data/output/{design}/type_dict_node_{file}inst.pkl', 'wb') as f:
        pickle.dump(type_node_dict, f)

processing aes_DC_305_1_0
processing aes_DC_395_0_1
processing aes_DC_365_0_1
processing aes_DC_400_1_1


KeyboardInterrupt: 

In [ ]:
df_full = pd.read_csv(f"/home/zluo/data/processed/aes/full_data.csv")
df_full['fl_uti'] = df_full['Variant'].apply(lambda name: int(name.split("_")[-1]))
base_mlp_df = df_full[['Wirelength', 'Util', 'target_clk_period_x', 'num_inst', 'num_net', 'avg_deg', 'num_PI', 'num_PO',
       'num_bTerms', 'in_deg_dis', 'out_deg_dis', 'num_seq','num_comb', 'opt_effort', 're_time', 'fl_uti']]

raw_X = base_mlp_df.iloc[:,1:].to_numpy()
Y = base_mlp_df.iloc[:,0].to_numpy()

X = []
for x in raw_X:
    new_x = []
    for entry in x:
        if type(entry) == str:
            lst = entry[1:-1].split('.')[:-1]
            new_lst = [float(val) for val in lst]
            new_x += new_lst
        else:
            new_x.append(float(entry))
    X.append(new_x)
    
X_global = np.array(X)
Y_global = np.array(Y)

np.save("data/X_global.npy", X_global)
np.save("data/Y_global.npy", Y_global)

In [ ]:
all_zero_h_lst = []
all_one_h_lst = []

file_lst = os.listdir(f"/home/zluo/data/graphs/{design}")

for path in file_lst:
    file = path.split(".")[0]
    zero_h_lst = []
    one_h_lst = []
    pd_local_lst = np.load(f"/home/zluo/data/processed/{design}/local_pd/pd_local_lst_{file}.npy", allow_pickle=True)
    for dgms in pd_local_lst:
        zero_h = np.array([list([tp[1][-1], tp[1][0]]) for tp in dgms[0]] + [list([tp[1][-1], tp[1][0]]) for tp in dgms[2]])
        one_h = np.array([list([tp[1][-1], tp[1][0]]) for tp in dgms[-1]])
        if len(zero_h) == 0:
            zero_h = np.array([[0., 0.]])

        if len(one_h) == 0:
            one_h = np.array([[0., 0.]])

        all_zero_h_lst.append(zero_h)
        all_one_h_lst.append(one_h)
        
pimgr_0 = PersistenceImager(pixel_size=1)
pimgr_0.fit(all_zero_h_lst)

pimgr_1 = PersistenceImager(pixel_size=1)
pimgr_1.fit(all_one_h_lst)

file_lst = os.listdir(f"/home/zluo/data/graphs/{design}")

for path in file_lst:
    file = path.split(".")[0]
    zero_h_lst = []
    one_h_lst = []
    pd_local_lst = np.load(f"/home/zluo/data/processed/{design}/local_pd/pd_local_lst_{file}.npy", allow_pickle=True)
    for dgms in pd_local_lst:
        zero_h = np.array([list([tp[1][-1], tp[1][0]]) for tp in dgms[0]] + [list([tp[1][-1], tp[1][0]]) for tp in dgms[2]])
        one_h = np.array([list([tp[1][-1], tp[1][0]]) for tp in dgms[-1]])
        if len(zero_h) == 0:
            zero_h = np.array([[0., 0.]])

        if len(one_h) == 0:
            one_h = np.array([[0., 0.]])
        
        zero_h_lst.append(zero_h)
        one_h_lst.append(one_h)
        
    L_0 = pimgr_0.transform(zero_h_lst)
    L_1 = pimgr_1.transform(one_h_lst)
    
    L_0 = [img.flatten() for img in L_0]
    L_1 = [img.flatten() for img in L_1]
    
    np.save(f"/home/zluo/data/processed/{design}/local_pd/L_0_{file}.npy", L_0)
    np.save(f"/home/zluo/data/processed/{design}/local_pd/L_1_{file}.npy", L_1)

In [3]:
dataset = netlist("/home/zluo/nn/gnn/")
#dataset.process()

In [ ]:
dataset_pd = netlist_pd("/home/zluo/nn/gnn2/")
#dataset_pd.process()

In [10]:
dataset_pd = netlist_gb("/home/zluo/nn/gnn3/")
#dataset_pd.process()

Processing...


1833
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

Done!


In [159]:
dataset = dataset_pd

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(32, 35, 3).to(device)
#model = GAT(35, 3, 8, 8, 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
train_idx = np.load("data/X_train_idx.npy", allow_pickle=True)
test_idx = np.load("data/X_test_idx.npy", allow_pickle=True)
train_dataset = dataset[train_idx]
test_dataset = dataset[test_idx]

In [6]:
def square_relative_error(output, target):
    loss = ((output - target)**2)/((target**2))
    return loss

def mean_relative_error(output, target):
    loss = abs(output - target)/abs(target)
    return loss

def mean_abs_error(output, target):
    loss = abs(output - target)
    return loss

def mse(output, target):
    loss = (output - target)**2
    return loss

In [7]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
criterion = square_relative_error
eval_crs = [square_relative_error, mean_relative_error, mean_abs_error, mse]

In [8]:
loss_train_all = []
loss_val_all = []
n = 100
for epoch in range(n):
    model.train()

    loss_train_tp = [[] for i in range(len(eval_crs))]
    for data in train_loader:
        data = data.to(device)
        true_y = torch.FloatTensor(data.y).reshape(-1, 1)
        true_y = true_y.to(device)
        optimizer.zero_grad()
        if gb:
            output = model(data.x, data.edge_index, data.batch)
        else:
            output = model(data.x, data.edge_index, data.batch, data.stats)
        loss = criterion(output, true_y)

        for idx in range(len(eval_crs)):
            cri = eval_crs[idx]
            loss_tr = cri(output, true_y)
            loss_train_tp[idx] += loss_tr.flatten().tolist()

        loss.sum().backward()
        optimizer.step()
    loss_train_tp = np.mean(np.array(loss_train_tp), axis=1)


    model.eval()


    loss_val_tp = [[] for i in range(len(eval_crs))]
    for data in test_loader:
        data = data.to(device)
        true_y = torch.FloatTensor(data.y).reshape(-1, 1)
        true_y = true_y.to(device)
        if gb:
            output = model(data.x, data.edge_index, data.batch)
        else:
            output = model(data.x, data.edge_index, data.batch, data.stats)

        for idx in range(len(eval_crs)):
            cri = eval_crs[idx]
            loss_t = cri(output, true_y)
            loss_val_tp[idx] += loss_t.flatten().tolist()

    loss_val_tp = np.mean(np.array(loss_val_tp), axis=1)
    #with open("train_log", "w", buffering=1) as f:
    print(f"{epoch}, {loss_train_tp}, {loss_val_tp}")

    loss_train_all.append(loss_train_tp)
    loss_val_all.append(loss_val_tp)

NameError: name 'gb' is not defined

In [11]:
from torch.optim.lr_scheduler import StepLR

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.1)

In [13]:
train_result, val_result = experiment(1000, train_dataset, test_dataset, model, criterion, optimizer, scheduler, device, eval_crs)

0, [3.86812878e-03 5.05132583e-02 3.11621814e+03 1.50786585e+07], [3.07247033e-03 4.79763940e-02 2.89539576e+03 1.10675551e+07]
1, [3.73368641e-03 4.96877106e-02 3.06412557e+03 1.45310400e+07], [3.00887481e-03 4.74773792e-02 2.86509536e+03 1.08358513e+07]
2, [3.64654798e-03 4.90881104e-02 3.02713438e+03 1.41916112e+07], [2.95567745e-03 4.70631586e-02 2.83945362e+03 1.06349259e+07]
3, [3.55937258e-03 4.84850301e-02 2.98997641e+03 1.38533674e+07], [2.91361715e-03 4.67361776e-02 2.81873766e+03 1.04695080e+07]
4, [3.47021900e-03 4.78774846e-02 2.95255348e+03 1.35074937e+07], [2.87039441e-03 4.63899759e-02 2.79707361e+03 1.03033443e+07]
5, [3.39278150e-03 4.73278673e-02 2.91866059e+03 1.32063907e+07], [2.81807095e-03 4.59629277e-02 2.77098385e+03 1.01108493e+07]
6, [3.31327578e-03 4.67584756e-02 2.88345645e+03 1.28948423e+07], [2.73894829e-03 4.53017157e-02 2.73096027e+03 9.82492287e+06]
7, [3.22496622e-03 4.60902128e-02 2.84205269e+03 1.25462437e+07], [2.65584002e-03 4.45907612e-02 2.68805

64, [1.10800301e-03 2.68996110e-02 1.64792033e+03 4.19762253e+06], [6.18526624e-04 1.98643321e-02 1.23173215e+03 2.45227837e+06]
65, [1.10457015e-03 2.68569236e-02 1.64524645e+03 4.18410300e+06], [6.14889085e-04 1.98089055e-02 1.22790953e+03 2.43496842e+06]
66, [1.10107527e-03 2.68143102e-02 1.64258288e+03 4.17039902e+06], [6.11189090e-04 1.97527391e-02 1.22400131e+03 2.41713064e+06]
67, [1.09772880e-03 2.67735070e-02 1.64003014e+03 4.15729898e+06], [6.07806778e-04 1.97017971e-02 1.22044204e+03 2.40074993e+06]
68, [1.09431403e-03 2.67316972e-02 1.63741908e+03 4.14395139e+06], [6.04251727e-04 1.96481732e-02 1.21665788e+03 2.38323209e+06]
69, [1.09097446e-03 2.66906620e-02 1.63485151e+03 4.13087740e+06], [6.01101938e-04 1.95998823e-02 1.21325178e+03 2.36766826e+06]
70, [1.08768083e-03 2.66502861e-02 1.63232973e+03 4.11799992e+06], [5.98146252e-04 1.95561633e-02 1.21014229e+03 2.35296751e+06]
71, [1.08536965e-03 2.66241089e-02 1.63068124e+03 4.10885040e+06], [5.94956254e-04 1.95101965e-02

128, [5.97149155e-04 1.95727850e-02 1.19697555e+03 2.25595908e+06], [5.16824385e-04 1.81690517e-02 1.10692992e+03 1.91656058e+06]
129, [5.96953827e-04 1.95684506e-02 1.19666601e+03 2.25490463e+06], [5.16751459e-04 1.81669811e-02 1.10673594e+03 1.91586895e+06]
130, [5.96769864e-04 1.95644087e-02 1.19637852e+03 2.25392746e+06], [5.16680998e-04 1.81649564e-02 1.10655068e+03 1.91522459e+06]
131, [5.96590709e-04 1.95604841e-02 1.19610238e+03 2.25299911e+06], [5.16613234e-04 1.81629830e-02 1.10637397e+03 1.91462511e+06]
132, [5.96418374e-04 1.95568844e-02 1.19585005e+03 2.25212490e+06], [5.16545249e-04 1.81610202e-02 1.10620314e+03 1.91405824e+06]
133, [5.96252357e-04 1.95534138e-02 1.19560918e+03 2.25130073e+06], [5.16482483e-04 1.81591641e-02 1.10604251e+03 1.91353428e+06]
134, [5.96090091e-04 1.95500061e-02 1.19537506e+03 2.25051251e+06], [5.16420357e-04 1.81574905e-02 1.10589795e+03 1.91303827e+06]
135, [5.95933338e-04 1.95467065e-02 1.19515033e+03 2.24976577e+06], [5.16357209e-04 1.8155

192, [5.89388550e-04 1.94245718e-02 1.18754113e+03 2.22467408e+06], [5.12632549e-04 1.80808836e-02 1.10065211e+03 1.89572182e+06]
193, [5.89291120e-04 1.94228559e-02 1.18743718e+03 2.22432271e+06], [5.12562891e-04 1.80796722e-02 1.10057624e+03 1.89545847e+06]
194, [5.89193429e-04 1.94211424e-02 1.18733344e+03 2.22397041e+06], [5.12491663e-04 1.80784385e-02 1.10049929e+03 1.89519085e+06]
195, [5.89096479e-04 1.94194467e-02 1.18723067e+03 2.22362044e+06], [5.12420691e-04 1.80772104e-02 1.10042274e+03 1.89492460e+06]
196, [5.88999019e-04 1.94177504e-02 1.18712785e+03 2.22326790e+06], [5.12346478e-04 1.80759160e-02 1.10034273e+03 1.89464984e+06]
197, [5.88902705e-04 1.94160621e-02 1.18702549e+03 2.22291985e+06], [5.12276679e-04 1.80747136e-02 1.10026762e+03 1.89438701e+06]
198, [5.88805715e-04 1.94143708e-02 1.18692304e+03 2.22256965e+06], [5.12204493e-04 1.80734647e-02 1.10019005e+03 1.89411771e+06]
199, [5.88709621e-04 1.94126933e-02 1.18682129e+03 2.22222178e+06], [5.12131076e-04 1.8072

256, [5.34105318e-04 1.84341613e-02 1.12681029e+03 2.01356947e+06], [5.01544986e-04 1.79034803e-02 1.09363368e+03 1.87828350e+06]
257, [5.34094000e-04 1.84338628e-02 1.12678913e+03 2.01350868e+06], [5.01533769e-04 1.79031370e-02 1.09361006e+03 1.87822431e+06]
258, [5.34082884e-04 1.84335700e-02 1.12676833e+03 2.01344890e+06], [5.01524636e-04 1.79028329e-02 1.09358864e+03 1.87817187e+06]
259, [5.34071571e-04 1.84332745e-02 1.12674745e+03 2.01338881e+06], [5.01514386e-04 1.79025208e-02 1.09356687e+03 1.87811612e+06]
260, [5.34060299e-04 1.84329874e-02 1.12672719e+03 2.01332952e+06], [5.01504564e-04 1.79022118e-02 1.09354523e+03 1.87806179e+06]
261, [5.34049049e-04 1.84326988e-02 1.12670686e+03 2.01327034e+06], [5.01494733e-04 1.79019053e-02 1.09352402e+03 1.87800892e+06]
262, [5.34037426e-04 1.84323997e-02 1.12668590e+03 2.01320976e+06], [5.01484545e-04 1.79015897e-02 1.09350205e+03 1.87795352e+06]
263, [5.34025801e-04 1.84321048e-02 1.12666517e+03 2.01314914e+06], [5.01474608e-04 1.7901

321, [5.27206228e-04 1.83082714e-02 1.11944406e+03 1.98884998e+06], [5.01372785e-04 1.78986215e-02 1.09357472e+03 1.87900399e+06]
322, [5.27204978e-04 1.83082431e-02 1.11944227e+03 1.98884487e+06], [5.01371521e-04 1.78985865e-02 1.09357253e+03 1.87899900e+06]
323, [5.27203717e-04 1.83082118e-02 1.11944030e+03 1.98883986e+06], [5.01370349e-04 1.78985620e-02 1.09357102e+03 1.87899445e+06]
324, [5.27202577e-04 1.83081861e-02 1.11943869e+03 1.98883539e+06], [5.01369247e-04 1.78985371e-02 1.09356944e+03 1.87899000e+06]
325, [5.27201253e-04 1.83081569e-02 1.11943684e+03 1.98883006e+06], [5.01367916e-04 1.78985105e-02 1.09356773e+03 1.87898444e+06]
326, [5.27200103e-04 1.83081312e-02 1.11943525e+03 1.98882573e+06], [5.01367495e-04 1.78984958e-02 1.09356681e+03 1.87898275e+06]
327, [5.27198882e-04 1.83081050e-02 1.11943359e+03 1.98882083e+06], [5.01366624e-04 1.78984748e-02 1.09356552e+03 1.87897947e+06]
328, [5.27197679e-04 1.83080758e-02 1.11943172e+03 1.98881587e+06], [5.01365052e-04 1.7898

KeyboardInterrupt: 